In [1]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [5]:
from ultralytics import YOLO

In [6]:
# pip install dill

Note: you may need to restart the kernel to use updated packages.


In [7]:
model = YOLO('best.pt')

In [8]:
model.info()

YOLOv8n-seg summary: 261 layers, 3263811 parameters, 0 gradients


(261, 3263811, 0, 0.0)

In [9]:
# model.train(data='./dataset/data.yaml', epochs=5, imgsz=640) 

In [20]:
# Assuming 'model' is your trained YOLO model
results = model.val(data='./dataset/data.yaml')

Ultralytics YOLOv8.2.14  Python-3.12.3 torch-2.3.0+cpu CPU (Intel Core(TM) i5-8265U 1.60GHz)


val: Scanning C:\Users\Kishan.Ajudiya\Desktop\thesis\Lawn_Segmentation\dataset\val\labels.cache... 9 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9/9 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:03<00:00,  3.38s/it]


                   all          9         10          1      0.663      0.783      0.642          1      0.663      0.782      0.591
Speed: 3.7ms preprocess, 248.0ms inference, 0.0ms loss, 12.8ms postprocess per image
Results saved to runs\segment\val2


In [21]:
print(results)


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001C73E716AB0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.0410

In [23]:
test = model.val(data='./dataset/data.yaml', task='test')
print(test)

Ultralytics YOLOv8.2.14  Python-3.12.3 torch-2.3.0+cpu CPU (Intel Core(TM) i5-8265U 1.60GHz)


val: Scanning C:\Users\Kishan.Ajudiya\Desktop\thesis\Lawn_Segmentation\dataset\val\labels.cache... 9 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9/9 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:03<00:00,  3.23s/it]


                   all          9         10          1      0.663      0.783      0.642          1      0.663      0.782      0.591
Speed: 4.5ms preprocess, 230.1ms inference, 0.0ms loss, 11.2ms postprocess per image
Results saved to runs\test\val2
ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001C73EB4C830>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022, 

In [24]:
# Set the test flag to True
test_results = model.val(data='./dataset/data.yaml', task='test')
print(test_results)


Ultralytics YOLOv8.2.14  Python-3.12.3 torch-2.3.0+cpu CPU (Intel Core(TM) i5-8265U 1.60GHz)


val: Scanning C:\Users\Kishan.Ajudiya\Desktop\thesis\Lawn_Segmentation\dataset\test\labels... 1 images, 38 backgrounds, 0 corrupt: 100%|██████████| 39/39 [00:00<00:00, 262.94it/s]

val: New cache created: C:\Users\Kishan.Ajudiya\Desktop\thesis\Lawn_Segmentation\dataset\test\labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:18<00:00,  6.04s/it]


                   all         39          1     0.0256          1     0.0415     0.0332     0.0256          1     0.0415     0.0332
Speed: 4.1ms preprocess, 311.3ms inference, 0.0ms loss, 17.9ms postprocess per image
Results saved to runs\test\val3
ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001C73A011C40>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022, 